This is a slight variation of tutorial 1
to test camera issues between different versions of Carla
Version 9.14 had a problem on Windows where in non-standard
aspect ratios it was scrambling camera image.
This code is to test for these.
Makes sure you have mutiple version of Carla installed
so you can pick specific kernels for this notebook to run
different Carla versions.
Instructions:
1. Open v9.14 Sim and select v9.14 kernel
2. Run all steps
3. See if camera windows looks good
4. Do all of the above steps with v9.15

And do not forget to re-launch different Sim for that version

In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import math

import numpy as np #in this example to change image dddrepresentation - re-shaping

In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)

In [4]:
# define environment/world and get possible places to spawn a car
world = client.load_world('Town02_opt')
world.unload_map_layer(carla.MapLayer.All)
# world.unload_map_layer(carla.MapLayer.Buildings)
# world.unload_map_layer(carla.MapLayer.Decals)
# world.unload_map_layer(carla.MapLayer.Foliage)
# world.unload_map_layer(carla.MapLayer.Props)
spawn_points = world.get_map().get_spawn_points()

In [5]:
# world = client.load_world('Town01')
# spawn_points = world.get_map().get_spawn_points()

In [6]:
list_actor = world.get_actors()
for actor_ in list_actor:
    if isinstance(actor_, carla.TrafficLight):
        # for any light, first set the light state, then set time. for yellow it is 
        # carla.TrafficLightState.Yellow and Red it is carla.TrafficLightState.Red
        actor_.set_state(carla.TrafficLightState.Green) 
        actor_.set_green_time(1000.0)
            # actor_.set_green_time(5000.0)
            # actor_.set_yellow_time(1000.0)

In [7]:
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [8]:
#spawn a car in a random location

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)


In [9]:
# move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+3 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)

In [10]:
#send the car off on autopilot - this will leave the spectator
vehicle.set_autopilot(True)

In [11]:
#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

#camera mount offset on the car - you can tweak these to each car to avoid any parts of the car being in the view
CAMERA_POS_Z = 1.6 #this means 1.6m up from the ground
CAMERA_POS_X = 0.9 #this is 0.9m forward

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # 16:9 type ratios work in CARLA 9.14 on Windows so we change it to something different here
camera_bp.set_attribute('image_size_y', '360')
camera_bp.set_attribute('fov', '90')
# 1920
# 1080
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

# def camera_callback(image,data_dict):
#     data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

def camera_callback(image, data_dict):
    # Assuming image is RGBA, convert it to RGB
    image_rgb = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))[:, :, :3]
    data_dict['image'] = image_rgb

# Rest of your code remains the same


image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

In [12]:
''' 
This is the new Bit for tutorial 4
First we need to create controls functions so we could
push the car along the route
'''
# define speed contstants
PREFERRED_SPEED = 30 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1

def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 


In [13]:
from ultralytics import YOLO
model = YOLO(r"F:\projectfinal\80_epochs\runs\detect\train\weights\best.pt")

In [14]:
# while True:  
#     image = camera_data['image']
#     pre = model.predict(image, show=True)
#     cv2.imshow('RGB cameras', image)
#     if cv2.waitKey(1) == ord('q'):
#         breakw
# cv2.destroyAllWindows()

In [15]:
# # now little demo to drive straight
# # close to a desired speed

# # - press Q to exit, you need to run the bit above to start the car
# import math

# cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
# cv2.imshow('RGB Camera',camera_data['image'])

# #main loop 
# quit = False

# while True:
#     # Carla Tick
#     world.tick()
#     if cv2.waitKey(1) == ord('q'):
#         quit = True
#         break
#     image = camera_data['image']
#     pre = model.predict(image, show=True)
# #     cv2.imshow('RGB cameras', image)
#     steering_angle = 0 # we do not have it yet
#     # to get speed we need to use 'get velocity' function
#     v = vehicle.get_velocity()
#     # if velocity is a vector in 3d
#     # then speed is like hypothenuse in a right triangle
#     # and 3.6 is a conversion factor from meters per second to kmh
#     # e.g. kmh is 1000 meters and one hour is 60 min with 60 sec = 3600 sec
#     speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
#     # now we add the speed to the window showing a camera mounted on the car
#     image = cv2.putText(image, 'Speed: '+str(int(speed))+' kmh', org, 
#                         font, fontScale, color, thickness, cv2.LINE_AA)
#     # this is where we used the function above to determine accelerator input
#     # from current speed
#     estimated_throttle = maintain_speed(speed)
#     # now we apply accelerator
#     vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, 
#                                                steer=steering_angle))
#     cv2.imshow('RGB Camera',image)

# #clean up
# cv2.destroyAllWindows()
# camera.stop()
# for actor in world.get_actors().filter('*vehicle*'):
#     actor.destroy()
# for sensor in world.get_actors().filter('*sensor*'):
#     sensor.destroy()

In [16]:
# now little demo to drive straight
# close to a desired speed

# - press Q to exit, you need to run the bit above to start the car

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

#main loop 
quit = False

while True:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break
    image = camera_data['image']
    pre = model.predict(image, show=True)
    um_image = cv2.UMat(image)

#     cv2.imshow('RGB cameras', image)
    steering_angle = 0 # we do not have it yet
    # to get speed we need to use 'get velocity' function
    v = vehicle.get_velocity()
    # if velocity is a vector in 3d
    # then speed is like hypothenuse in a right triangle
    # and 3.6 is a conversion factor from meters per second to kmh
    # e.g. kmh is 1000 meters and one hour is 60 min with 60 sec = 3600 sec
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    # now we add the speed to the window showing a camera mounted on the car
    um_image = cv2.putText(um_image, 'Speed: '+str(int(speed))+' kmh', org, font, fontScale, color, thickness, cv2.LINE_AA)

    # this is where we used the function above to determine accelerator input
    # from current speed
    estimated_throttle = maintain_speed(speed)
    # now we apply accelerator
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, 
                                               steer=steering_angle))
    cv2.imshow('RGB Camera',um_image)

#clean up
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


0: 384x640 (no detections), 513.6ms
Speed: 3.0ms preprocess, 513.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 537.9ms
Speed: 6.0ms preprocess, 537.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 506.9ms
Speed: 7.0ms preprocess, 506.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 485.9ms
Speed: 5.0ms preprocess, 485.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 509.8ms
Speed: 5.0ms preprocess, 509.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 502.0ms
Speed: 4.0ms preprocess, 502.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 488.0ms
Speed: 5.0ms preprocess, 488.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 508.0ms
Speed: 4.0ms prepr


0: 384x640 (no detections), 520.1ms
Speed: 5.0ms preprocess, 520.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 528.0ms
Speed: 3.0ms preprocess, 528.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 503.0ms
Speed: 4.0ms preprocess, 503.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 519.0ms
Speed: 4.0ms preprocess, 519.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 512.0ms
Speed: 5.0ms preprocess, 512.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 525.0ms
Speed: 3.9ms preprocess, 525.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 508.3ms
Speed: 5.0ms preprocess, 508.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 514.7ms
Speed: 3.0ms prepr


0: 384x640 (no detections), 520.9ms
Speed: 5.2ms preprocess, 520.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 507.5ms
Speed: 4.0ms preprocess, 507.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 511.0ms
Speed: 4.0ms preprocess, 511.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 514.0ms
Speed: 3.0ms preprocess, 514.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 502.0ms
Speed: 5.0ms preprocess, 502.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 505.0ms
Speed: 4.0ms preprocess, 505.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 513.0ms
Speed: 4.0ms preprocess, 513.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 516.0ms
Speed: 4.0ms prepr

In [ ]:
# clean up after yourself

camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()